In [4]:
import pandas as pd

# Load the data
population = pd.read_csv("Population.csv")

# Clean column names: lowercase, replace hyphens with underscores
population.columns = (
    population.columns
    .str.lower()
    .str.replace("-", "_")
    .str.replace(" ", "_")
)

# Show cleaned columns
print("Cleaned columns:", population.columns.tolist())

# Confirm data types and nulls (should be fine)
print(population.info())
print(population.isnull().sum())

# Optional: Save cleaned version for PostgreSQL import
population.to_csv("p_cleaned.csv", index=False)


Cleaned columns: ['sa2_code', 'sa2_name', '0_4_people', '5_9_people', '10_14_people', '15_19_people', '20_24_people', '25_29_people', '30_34_people', '35_39_people', '40_44_people', '45_49_people', '50_54_people', '55_59_people', '60_64_people', '65_69_people', '70_74_people', '75_79_people', '80_84_people', '85_and_over_people', 'total_people']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   sa2_code            373 non-null    int64 
 1   sa2_name            373 non-null    object
 2   0_4_people          373 non-null    int64 
 3   5_9_people          373 non-null    int64 
 4   10_14_people        373 non-null    int64 
 5   15_19_people        373 non-null    int64 
 6   20_24_people        373 non-null    int64 
 7   25_29_people        373 non-null    int64 
 8   30_34_people        373 non-null    int64 
 9   35_39_people  

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon
from geoalchemy2 import Geometry, WKTElement
import matplotlib.pyplot as plt

In [48]:
# This loads the POIs from the REST service directly
poi_url = "https://maps.six.nsw.gov.au/arcgis/rest/services/public/NSW_POI/MapServer/0/query?where=1%3D1&outFields=*&f=geojson"

pois = gpd.read_file(poi_url)
print(pois.columns)
pois.head()

Index(['objectid', 'topoid', 'poigroup', 'poitype', 'poiname', 'poilabel',
       'poilabeltype', 'poialtlabel', 'poisourcefeatureoid', 'accesscontrol',
       'startdate', 'enddate', 'lastupdate', 'msoid', 'centroidid',
       'shapeuuid', 'changetype', 'processstate', 'urbanity', 'geometry'],
      dtype='object')


,objectid,topoid,poigroup,poitype,poiname,poilabel,poilabeltype,poialtlabel,poisourcefeatureoid,accesscontrol,startdate,enddate,lastupdate,msoid,centroidid,shapeuuid,changetype,processstate,urbanity,geometry
0,1,500000000,9,Mine - Underground,None,Mine - Underground,GENERIC,None,157,1,1628668563000,32503680000000,1628668617000,233046,None,729e2b57-0cd4-3f70-90fa-9dce09e34a8e,I,None,S,POINT (152.12202 -31.10616)
1,2,500005504,3,Lookout,KUNDERANG LOOKOUT,KUNDERANG LOOKOUT,NAMED,None,56,1,1285588392000,32503680000000,1285588392535,83091,None,d88a28a8-c572-3992-995f-d26a274aea18,I,None,S,POINT (152.29869 -31.02148)
2,3,500005505,3,Lookout,FALLS LOOKOUT,FALLS LOOKOUT,NAMED,None,56,1,1285588392000,32503680000000,1285588392535,83691,None,21b476d2-6519-3e28-8b19-1526fcb9652f,I,None,S,POINT (152.33786 -31.01576)
3,4,500005507,3,Lookout,MCCOYS LOOKOUT,MCCOYS LOOKOUT,NAMED,None,56,1,1285588392000,32503680000000,1285588392535,83380,None,016d69b2-6530-39e7-89a6-6e3054df55ac,I,None,S,POINT (152.34181 -31.01897)
4,5,500012781,3,Picnic Area,WILSON RIVER PICNIC AREA,WILSON RIVER PICNIC AREA,NAMED,None,62,1,1608714678000,32503680000000,1608714706360,231054,None,49ad26c8-609e-3aa0-b4ad-51459b43ab51,M,None,S,POINT (152.47882 -31.20754)


In [30]:
# Load stops.txt
stops = pd.read_csv('Stops.txt')

# Preview
stops.head()

# Some cleaning
# Drop rows where stop_lat or stop_lon is missing
stops = stops.dropna(subset=['stop_lat', 'stop_lon'])

# We want to only select the stops inside three areas: 
# Inner West, North Sydney and Hornsby, City and Inner South

# No column name telling which region a stop belongs to
# So: use the stop's latitude and longitude (stop_lat, stop_lon) to filter our disired data

In [42]:
sa2 = gpd.read_file("~/Desktop/DATA2001/DATA2001-A/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp")

print(sa2.columns)
sa2.head()

sa2_sydney = sa2[sa2['GCC_NAME21'] == 'Greater Sydney']

# Create geometry column for stops
stops['geometry'] = stops.apply(lambda row: Point(row['stop_lon'], row['stop_lat']), axis=1)

# Turn stops into a GeoDataFrame (same CRS)
stops_gdf = gpd.GeoDataFrame(stops, geometry='geometry', crs="EPSG:4326")

stops_gdf = stops_gdf.to_crs(epsg=7844)

stops_with_sa2 = gpd.sjoin(stops_gdf, sa2_sydney, how='left', predicate='within')

# print(stops_with_sa2.head(10)) #debug

target_regions = [
    'Sydney - Inner West',
    'Sydney - North Sydney and Hornsby',
    'Sydney - City and Inner South'
]

filtered_stops = stops_with_sa2[stops_with_sa2['SA4_NAME21'].isin(target_regions)]

filtered_stops = filtered_stops.dropna(subset=['SA4_NAME21'])

columns_to_keep = [
    'stop_id', 'stop_name', 'stop_lat', 'stop_lon',
    'wheelchair_boarding', 'SA2_CODE21', 'SA2_NAME21', 'SA4_NAME21', 'geometry'
]

filtered_stops = filtered_stops[columns_to_keep]

filtered_stops.columns = filtered_stops.columns.str.lower()

filtered_stops.to_csv('s_filtered.csv', index=False)


Index(['SA2_CODE21', 'SA2_NAME21', 'CHG_FLAG21', 'CHG_LBL21', 'SA3_CODE21',
       'SA3_NAME21', 'SA4_CODE21', 'SA4_NAME21', 'GCC_CODE21', 'GCC_NAME21',
       'STE_CODE21', 'STE_NAME21', 'AUS_CODE21', 'AUS_NAME21', 'AREASQKM21',
       'LOCI_URI21', 'geometry'],
      dtype='object')


In [14]:
# Adjust path as needed
catchments_primary = gpd.read_file("~/Desktop/DATA2001/DATA2001-A/catchments/catchments_primary.shp")

# Preview structure
print(catchments_primary.columns)
catchments_primary.head()

Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry'],
      dtype='object')


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,2838,PRIMARY,Parklea PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.93564 -33.71612, 150.93715 -33.7..."
1,2404,PRIMARY,Lindfield EPS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.18336 -33.74748, 151.18443 -33.7..."
2,4393,PRIMARY,Carlingford WPS,20220223,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.04518 -33.77303, 151.04526 -33.7..."
3,4615,PRIMARY,Caddies Ck PS,20181210,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((150.92567 -33.7296, 150.92602 -33.72..."
4,3918,PRIMARY,Killara PS,20211219,Y,Y,Y,Y,Y,Y,Y,N,N,N,N,N,N,None,"POLYGON ((151.15379 -33.75586, 151.15404 -33.7..."


In [18]:
# Adjust path as needed
catchments_secondary = gpd.read_file("~/Desktop/DATA2001/DATA2001-A/catchments/catchments_secondary.shp")

# Preview structure
print(catchments_secondary.columns)
catchments_secondary.head()

Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry'],
      dtype='object')


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,PRIORITY,geometry
0,8503,HIGH_COED,Billabong HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.67182 -35.31444, 146.6893 -35.31..."
1,8266,HIGH_COED,James Fallon HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((147.08734 -35.86271, 147.10413 -35.8..."
2,8505,HIGH_COED,Murray HS,20200507,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((146.81448 -35.78341, 146.8125 -35.79..."
3,8458,HIGH_COED,Kingswood HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"MULTIPOLYGON (((150.686 -33.74031, 150.68631 -..."
4,8559,HIGH_COED,Jamison HS,20201016,N,N,N,N,N,N,N,Y,Y,Y,Y,Y,Y,None,"POLYGON ((150.69513 -33.75627, 150.68936 -33.7..."


In [20]:
# Adjust path as needed
catchments_future = gpd.read_file("~/Desktop/DATA2001/DATA2001-A/catchments/catchments_future.shp")

# Preview structure
print(catchments_future.columns)
catchments_future.head()

Index(['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1',
       'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9',
       'YEAR10', 'YEAR11', 'YEAR12', 'geometry'],
      dtype='object')


,USE_ID,CATCH_TYPE,USE_DESC,ADD_DATE,KINDERGART,YEAR1,YEAR2,YEAR3,YEAR4,YEAR5,YEAR6,YEAR7,YEAR8,YEAR9,YEAR10,YEAR11,YEAR12,geometry
0,8416,HIGH_COED,Ku-ring-gai HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.19849 -33.5399, 151.19945 -33.54..."
1,8161,HIGH_BOYS,Randwick BHS,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.27152 -33.91402, 151.27152 -33.9..."
2,8539,HIGH_COED,SSC Blackwattle Bay,20220609,0,0,0,0,0,0,0,0,0,0,0,2024,2024,"POLYGON ((151.15292 -33.83939, 151.16144 -33.8..."
3,8400,HIGH_COED,St Ives HS,20230114,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.17794 -33.6982, 151.17859 -33.69..."
4,8555,HIGH_COED,Rose Bay SC,20200220,0,0,0,0,0,0,0,2024,2024,2024,2024,2024,2024,"POLYGON ((151.28072 -33.83287, 151.28095 -33.8..."


In [38]:
from sqlalchemy import create_engine, text
import psycopg2
import psycopg2.extras
import json
import os
import pandas as pd

credentials = os.path.expanduser("~/Desktop/DATA2001/Tutorial/Credentials.json")


def pgconnect(credential_filepath, db_schema="public"):
    with open(credential_filepath) as f:
        db_conn_dict = json.load(f)
        host       = db_conn_dict['host']
        db_user    = db_conn_dict['user']
        db_pw      = db_conn_dict['password']
        default_db = db_conn_dict['user']
        port       = db_conn_dict['port']
        try:
            db = create_engine(f'postgresql+psycopg2://{db_user}:{db_pw}@{host}:{port}/{default_db}', echo=False)
            conn = db.connect()
            print('Connected successfully.')
        except Exception as e:
            print("Unable to connect to the database.")
            print(e)
            db, conn = None, None
        return db,conn

def query(conn, sqlcmd, args=None, df=True):
    result = pd.DataFrame() if df else None
    try:
        if df:
            result = pd.read_sql_query(sqlcmd, conn, params=args)
        else:
            result = conn.execute(text(sqlcmd), args).fetchall()
            result = result[0] if len(result) == 1 else result
    except Exception as e:
        print("Error encountered: ", e, sep='\n')
    return result

In [40]:
db, conn = pgconnect(credentials)
conn.execute(text("set search_path to NSWFuel"))

Connected successfully.


In [45]:
import os
import geopandas as gpd
import pandas as pd
import json
from sqlalchemy import create_engine, text

# Set paths and database connection information
catchments_dir = 'catchments'  
db_username = 'postgres'
db_password = '1230'
db_host = 'localhost'
db_port = '5432'
db_name = 'g_assignment'


# Read optional metadata
try:
    with open(os.path.join(catchments_dir, 'catchment_sf_info.json'), 'r') as f:
        metadata = json.load(f)
    print("Metadata information:", metadata)
except:
    print("Metadata not found or could not be parsed")

# Read three types of school catchment areas
future_catchments = gpd.read_file(os.path.join(catchments_dir, 'catchments_future.shp'))
primary_catchments = gpd.read_file(os.path.join(catchments_dir, 'catchments_primary.shp'))
secondary_catchments = gpd.read_file(os.path.join(catchments_dir, 'catchments_secondary.shp'))

# Add school type labels
future_catchments['school_type'] = 'Future'
primary_catchments['school_type'] = 'Primary'
secondary_catchments['school_type'] = 'Secondary'

# Check and adjust coordinate reference systems (CRS)
for gdf, name in [(future_catchments, 'Future'), (primary_catchments, 'Primary'), (secondary_catchments, 'Secondary')]:
    print(f"\n{name} catchment area data:")
    print("Shape:", gdf.shape)
    print("Columns:", gdf.columns.tolist())
    print("CRS:", gdf.crs)
    
    # Ensure the CRS is EPSG:4283
    if gdf.crs is None or str(gdf.crs) != 'EPSG:4283':
        if gdf.crs is None:
            print(f"Warning: {name} catchment area has no detected CRS, check the .prj file")
            # Try reading from the .prj file
            try:
                with open(os.path.join(catchments_dir, f'catchments_{name.lower()}.prj'), 'r') as f:
                    prj_content = f.read()
                print(f".prj file content found:{prj_content[:100]}...")
            except:
                print("Failed to read .prj file")
                gdf.set_crs(epsg=4326, inplace=True)  # Assume WGS 84 temporarily
        
        # Convert to EPSG:4283
        gdf = gdf.to_crs(epsg=4283)
        print(f"已将{name}覆盖区域转换为EPSG:4283")

# Merge all datasets
all_catchments = pd.concat([future_catchments, primary_catchments, secondary_catchments], ignore_index=True)
print("\nShape of merged data:", all_catchments.shape)

# Create database connection
engine = create_engine(f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}')

# Ensure PostGIS extension is enabled
with engine.connect() as conn:
    conn.execute(text("CREATE EXTENSION IF NOT EXISTS postgis"))  # Use text() for raw SQL
    conn.commit()  # Commit changes

# Import data into PostgreSQL
try:
    # Import merged table
    all_catchments.to_postgis(
        'school_catchments', 
        engine, 
        if_exists='replace',
        index=False
    )
    print("All school catchment data successfully imported into 'school_catchments' table")
    
    # Optionally import each type separately
    future_catchments.to_postgis('future_school_catchments', engine, if_exists='replace', index=False)
    primary_catchments.to_postgis('primary_school_catchments', engine, if_exists='replace', index=False)
    secondary_catchments.to_postgis('secondary_school_catchments', engine, if_exists='replace', index=False)
    print("三种类型的学校覆盖区域数据已分别导入到独立的表中")


except Exception as e:
    print(f"Error while importing data: {e}")

# Create spatial indexes
with engine.connect() as conn:
    table_names = ['school_catchments', 'future_school_catchments', 
                   'primary_school_catchments', 'secondary_school_catchments']
    
    for table in table_names:
        try:
            conn.execute(text(f"CREATE INDEX idx_{table}_geom ON {table} USING GIST (geometry)"))  # Use text()
            conn.commit()  # Commit changes
            print(f"Spatial index created for table '{table}'")
        except Exception as e:
            print(f"Error creating spatial index for '{table}': {e}")

# Verify import results
with engine.connect() as conn:
    for table in table_names:
        result = conn.execute(text(f"SELECT COUNT(*) FROM {table}")).fetchone()  # Use text()
        print(f"Table '{table}' has {result[0]} records")

Metadata information: {'current_enrolment_year': 2023}

Future catchment area data:
Shape: (30, 19)
Columns: ['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9', 'YEAR10', 'YEAR11', 'YEAR12', 'geometry', 'school_type']
CRS: EPSG:4283

Primary catchment area data:
Shape: (1662, 20)
Columns: ['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9', 'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry', 'school_type']
CRS: EPSG:4283

Secondary catchment area data:
Shape: (436, 20)
Columns: ['USE_ID', 'CATCH_TYPE', 'USE_DESC', 'ADD_DATE', 'KINDERGART', 'YEAR1', 'YEAR2', 'YEAR3', 'YEAR4', 'YEAR5', 'YEAR6', 'YEAR7', 'YEAR8', 'YEAR9', 'YEAR10', 'YEAR11', 'YEAR12', 'PRIORITY', 'geometry', 'school_type']
CRS: EPSG:4283

Shape of merged data: (2128, 20)


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5432 failed: FATAL:  password authentication failed for user "postgres"

(Background on this error at: https://sqlalche.me/e/20/e3q8)